# 编码器写太长了，分出来写解码器吧

也可以再复习一下整个的过程

In [27]:
# 前面的其实就到完成了MultiHeadAttention的向前传播的
# 这块用来把数据给接上，把数据集读出来并得到编码器跑出来的结果，作为下一步的解码器的需要

from datasets import IndexedInputTargetTranslationDataset
import torch
from torch.utils.data import DataLoader
from utils.pipe import input_target_collate_fn

config={
    'data_dir':"data/example/processed",
    'save_config':"checkpoints/example_config.json",
    'save_checkpoint':"checkpoints/example_model.pth",
    'save_log':"logs/example.log",
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'print_every': 1,
    'save_every': 1,
    'vocabulary_size': None,
    'd_model': 128,
    'layers_count': 1,
    'heads_count': 2,
    'd_ff': 128,
    'dropout_prob': 0.1,
    'label_smoothing': 0.1,
    'optimizer': "Adam",
    'lr': 0.001,
    'batch_size': 64,
    'epochs': 100,
    'positional_encoding': True,
    'clip_grads': False,
    'dataset_limit':None
}

train_dataset = IndexedInputTargetTranslationDataset(
    data_dir=config['data_dir'],
    phase='train',
    vocabulary_size=config['vocabulary_size'],
    limit=config['dataset_limit'])

val_dataset = IndexedInputTargetTranslationDataset(
    data_dir=config['data_dir'],
    phase='val',
    vocabulary_size=config['vocabulary_size'],
    limit=config['dataset_limit'])

train_dataloader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
#     shuffle=True, #研究怎么算的时候，就简单点，不随机了，而且还只取一小块来算
    shuffle=False,
    collate_fn=input_target_collate_fn)
#DataLoader的collate_fn函数是来实现序列padding的操作。#输入是batch，输出是batch中的tensor数据

val_dataloader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    collate_fn=input_target_collate_fn)

#因为前面生成的是迭代器，所以这么去取一个batch的数据
import itertools

n_items = 1  # 希望只取第一项
batch_iter = iter(train_dataloader)
for i, batch in itertools.islice(enumerate(batch_iter), n_items):
    data_ba1=batch

##参考这个
# class Transformer(nn.Module):

#     def __init__(self, encoder, decoder):
#         super(Transformer, self).__init__()

#         self.encoder = encoder
#         self.decoder = decoder

#     def forward(self, sources, inputs):
#         # sources : (batch_size, sources_len)
#         # inputs : (batch_size, targets_len - 1)
#         batch_size, sources_len = sources.size()
#         batch_size, inputs_len = inputs.size()

#         sources_mask = pad_masking(sources, sources_len)
#         memory_mask = pad_masking(sources, inputs_len)
#         inputs_mask = subsequent_masking(inputs) | pad_masking(inputs, inputs_len)

#         memory = self.encoder(sources, sources_mask)  # (batch_size, seq_len, d_model)
#         outputs, state = self.decoder(inputs, memory, memory_mask, inputs_mask)  # (batch_size, seq_len, d_model)
#         return outputs

from models import TransformerEncoder
from embeddings import PositionalEncoding
from dictionaries import IndexDictionary

source_dictionary = IndexDictionary.load(config['data_dir'], mode='source', vocabulary_size=config['vocabulary_size'])
target_dictionary = IndexDictionary.load(config['data_dir'], mode='target', vocabulary_size=config['vocabulary_size'])

source_vocabulary_size,target_vocabulary_size=source_dictionary.vocabulary_size, target_dictionary.vocabulary_size

source_embedding = PositionalEncoding(
    num_embeddings=source_vocabulary_size,
    embedding_dim=config['d_model'],
    dim=config['d_model'])  # why dim?
target_embedding = PositionalEncoding(
    num_embeddings=target_vocabulary_size,
    embedding_dim=config['d_model'],
    dim=config['d_model'])  # why dim?

encoder = TransformerEncoder(
    layers_count=config['layers_count'],
    d_model=config['d_model'],
    heads_count=config['heads_count'],
    d_ff=config['d_ff'],
    dropout_prob=config['dropout_prob'],
    embedding=source_embedding)

#还缺个mask
from utils.pad import pad_masking

sources=batch[0]
batch_size, sources_len = sources.size()
sources_mask = pad_masking(sources, sources_len)

memory = encoder(sources, sources_mask)#这里，是把这个结果叫做memory，一种记忆
memory.shape#memory就是那一个箭号了

torch.Size([64, 46, 128])

In [38]:
#        outputs, state = self.decoder(inputs, memory, memory_mask, inputs_mask)
#到这一步了，我接下来需要理解的就是这段的代码，这个decoder具体是做了什么，只有有了更加明晰的概念，才能逐渐明白这一切

#继续啃吧，孤独还是生活的主旋律，没有人会一直陪你聊天的，而且还是在聊你自己研究领域的模型，更多的是已读不回，
#更常见的是收拾自己的沮丧，表达出没关系的感觉，始终有着遥远的距离感，可望而不可即。
import torch
from torch import nn
from models import Sublayer,LayerNormalization,PointwiseFeedForwardNetwork
class TransformerDecoder(nn.Module):

    def __init__(self, layers_count, d_model, heads_count, d_ff, dropout_prob, embedding):
        #这里的d_ff就是PointwiseFeedForwardNetwork中间的层，做一点中间的变化罢了。其实并不那么重要，值为128，
        #只是帮助网络变得复杂一点罢了
        super(TransformerDecoder, self).__init__()

        self.d_model = d_model#就是128
        self.embedding = embedding#德语做的嵌入层
        self.decoder_layers = nn.ModuleList(
            [TransformerDecoderLayer(d_model, heads_count, d_ff, dropout_prob) for _ in range(layers_count)]
        )#译码器的主要的层
        #heads_count值为2，d_ff为128
        self.generator = nn.Linear(embedding.embedding_dim, embedding.num_embeddings)
        #这两个值为128与 35820，看上去就是一层映射，映射为生成的德语词的概率
        self.generator.weight = self.embedding.weight#改为嵌入层的权重
        """
        这里线性层的权重改成了嵌入层的权重，那么在向前传播的运算当中的线性层运算就是在用嵌入层的权重来计算
        在这段代码中，`self.embedding`是输入词汇的嵌入层，而`self.generator`是输出词汇的线性层（也可以称为投影层），
        在训练过程中，根据损失函数计算的梯度，会先优化`self.generator`的权重，再更新`self.embedding`的权重，
        因此嵌入层的权重确实被优化了两次。这种方法通常被称为共享嵌入层（shared embedding）。
        这种做法有助于减少模型参数的数量，以及提高其泛化能力。
        """

    def forward(self, inputs, memory, memory_mask, inputs_mask=None, state=None):
        # inputs: (batch_size, seq_len - 1, d_model)
        # memory: (batch_size, seq_len, d_model)

        inputs = self.embedding(inputs)#这里关注的是inputs，这个输入应该是掩码后的含有开始符的德文内容
        #经过嵌入层后张成128维的嵌入层数据
        # if state is not None:
        #     inputs = torch.cat([state.previous_inputs, inputs], dim=1)
        #
        #     state.previous_inputs = inputs

        for layer_index, decoder_layer in enumerate(self.decoder_layers):#啊哈，为什么这里有两个东西，完全不理解，23333
            # 哦哦哦，忘记了，才看到是enumerate迭代器，这种迭代器的写法，会返回两个东西，一个是索引的序号，另一个才是本身的内容
            if state is None:
                inputs = decoder_layer(inputs, memory, memory_mask, inputs_mask)# 就用到这个decoder本身，就跑了这一步
            else: # Use cache
                layer_cache = state.layer_caches[layer_index]# 呃，这里用上了layer_index
                # print('inputs_mask', inputs_mask)
                inputs = decoder_layer(inputs, memory, memory_mask, inputs_mask, layer_cache)

                state.update_state(
                    layer_index=layer_index,
                    layer_mode='self-attention',
                    key_projected=decoder_layer.self_attention_layer.sublayer.key_projected,
                    value_projected=decoder_layer.self_attention_layer.sublayer.value_projected,
                )
                state.update_state(
                    layer_index=layer_index,
                    layer_mode='memory-attention',
                    key_projected=decoder_layer.memory_attention_layer.sublayer.key_projected,
                    value_projected=decoder_layer.memory_attention_layer.sublayer.value_projected,
                )

        generated = self.generator(inputs)  # (batch_size, seq_len, vocab_size)
        #把译码器的结果映射为德语词向量大小的那啥的维度，估计是走一步归一化，然后变为每个词的概率
        return generated, state #state还没有看到有什么作用

    def init_decoder_state(self, **args):
        return DecoderState()

class TransformerDecoderLayer(nn.Module):

    def __init__(self, d_model, heads_count, d_ff, dropout_prob):
        super(TransformerDecoderLayer, self).__init__()
        self.self_attention_layer = Sublayer(MultiHeadAttention(heads_count, d_model, dropout_prob, mode='self-attention'), d_model)
        #就只有模式的区别
        self.memory_attention_layer = Sublayer(MultiHeadAttention(heads_count, d_model, dropout_prob, mode='memory-attention'), d_model)
        #这个不太一样，用的是记忆注意力，有点没研究过，还得把多头注意力摆出来研究研究
        self.pointwise_feedforward_layer = Sublayer(PointwiseFeedForwardNetwork(d_ff, d_model, dropout_prob), d_model)

    def forward(self, inputs, memory, memory_mask, inputs_mask, layer_cache=None):#这里好像有cache，但有一说一，没看到cache的调用
        #应该正常时没有用到这一块的cache的，就不用管了
        # print('self attention')
        # print('inputs_mask', inputs_mask)
        inputs = self.self_attention_layer(inputs, inputs, inputs, inputs_mask, layer_cache)#自注意力
        # print('memory attention')
        inputs = self.memory_attention_layer(inputs, memory, memory, memory_mask, layer_cache)#记忆注意力
        inputs = self.pointwise_feedforward_layer(inputs)#非线性变化
        return inputs


class MultiHeadAttention(nn.Module):

    def __init__(self, heads_count, d_model, dropout_prob, mode='self-attention'):
        super(MultiHeadAttention, self).__init__()

        assert d_model % heads_count == 0
        assert mode in ('self-attention', 'memory-attention')

        self.d_head = d_model // heads_count
        self.heads_count = heads_count
        self.mode = mode
        self.query_projection = nn.Linear(d_model, heads_count * self.d_head)#q矩阵
        self.key_projection = nn.Linear(d_model, heads_count * self.d_head)  #k矩阵
        self.value_projection = nn.Linear(d_model, heads_count * self.d_head)#v矩阵
        self.final_projection = nn.Linear(d_model, heads_count * self.d_head)
        self.dropout = nn.Dropout(dropout_prob)
        self.softmax = nn.Softmax(dim=3)

        self.attention = None
        # For cache
        self.key_projected = None
        self.value_projected = None

    def forward(self, query, key, value, mask=None, layer_cache=None):
        """

        Args:
            query: (batch_size, query_len, model_dim)
            key: (batch_size, key_len, model_dim)
            value: (batch_size, value_len, model_dim)
            mask: (batch_size, query_len, key_len)
            state: DecoderState
        """
        # print('attention mask', mask)
        batch_size, query_len, d_model = query.size()

        d_head = d_model // self.heads_count

        query_projected = self.query_projection(query)
        # print('query_projected', query_projected.shape)
        if layer_cache is None or layer_cache[self.mode] is None:  # Don't use cache
            #应该还是用的这个，因为没看到layer_cache
            key_projected = self.key_projection(key) 
            value_projected = self.value_projection(value)
        else:  # Use cache
            if self.mode == 'self-attention':#自注意力机制
                key_projected = self.key_projection(key)
                value_projected = self.value_projection(value)

                key_projected = torch.cat([key_projected, layer_cache[self.mode]['key_projected']], dim=1)#嘶，用上了拼接，这块没研究过
                value_projected = torch.cat([value_projected, layer_cache[self.mode]['value_projected']], dim=1)
            elif self.mode == 'memory-attention':#记忆注意力机制
                key_projected = layer_cache[self.mode]['key_projected']
                value_projected = layer_cache[self.mode]['value_projected']

        # For cache
        self.key_projected = key_projected
        self.value_projected = value_projected

        batch_size, key_len, d_model = key_projected.size()
        batch_size, value_len, d_model = value_projected.size()

        query_heads = query_projected.view(batch_size, query_len, self.heads_count, d_head).transpose(1, 2)  # (batch_size, heads_count, query_len, d_head)
        # print('query_heads', query_heads.shape)
        # print(batch_size, key_len, self.heads_count, d_head)
        # print(key_projected.shape)
        key_heads = key_projected.view(batch_size, key_len, self.heads_count, d_head).transpose(1, 2)  # (batch_size, heads_count, key_len, d_head)
        value_heads = value_projected.view(batch_size, value_len, self.heads_count, d_head).transpose(1, 2)  # (batch_size, heads_count, value_len, d_head)

        attention_weights = self.scaled_dot_product(query_heads, key_heads)  # (batch_size, heads_count, query_len, key_len)

        if mask is not None:
            # print('mode', self.mode)
            # print('mask', mask.shape)
            # print('attention_weights', attention_weights.shape)
            mask_expanded = mask.unsqueeze(1).expand_as(attention_weights)
            attention_weights = attention_weights.masked_fill(mask_expanded, -1e18)

        self.attention = self.softmax(attention_weights)  # Save attention to the object
        # print('attention_weights', attention_weights.shape)
        attention_dropped = self.dropout(self.attention)
        context_heads = torch.matmul(attention_dropped, value_heads)  # (batch_size, heads_count, query_len, d_head)
        # print('context_heads', context_heads.shape)
        context_sequence = context_heads.transpose(1, 2).contiguous()  # (batch_size, query_len, heads_count, d_head)
        context = context_sequence.view(batch_size, query_len, d_model)  # (batch_size, query_len, d_model)
        final_output = self.final_projection(context)
        # print('final_output', final_output.shape)

        return final_output
    
class DecoderState:

    def __init__(self):
        self.previous_inputs = torch.tensor([])
        self.layer_caches = defaultdict(lambda: {'self-attention': None, 'memory-attention': None})

    def update_state(self, layer_index, layer_mode, key_projected, value_projected):
        self.layer_caches[layer_index][layer_mode] = {
            'key_projected': key_projected,
            'value_projected': value_projected
        }

    # def repeat_beam_size_times(self, beam_size): # memory만 repeat하면 되는데 state에 memory는 넣지 않기로 했다.
    #     self.
    #     self.src = self.src.data.repeat(beam_size, 1)

    def beam_update(self, positions):
        for layer_index in self.layer_caches:
            for mode in ('self-attention', 'memory-attention'):
                if self.layer_caches[layer_index][mode] is not None:
                    for projection in self.layer_caches[layer_index][mode]:
                        cache = self.layer_caches[layer_index][mode][projection]
                        if cache is not None:
                            cache.data.copy_(cache.data.index_select(0, positions))


In [34]:
#译码器，满理解一下这个embedding，就从embedding开始理解这个译码器吧
decoder = TransformerDecoder(
    layers_count=config['layers_count'],
    d_model=config['d_model'],
    heads_count=config['heads_count'],
    d_ff=config['d_ff'],
    dropout_prob=config['dropout_prob'],
    embedding=target_embedding)
#借以参照初始化的参数

In [35]:
target_embedding #这个就是按照德语的词表建立一个嵌入层，只是数据集不一样罢了
#很快就能理解了，可以往下走

PositionalEncoding(
  (embbedding): Embedding(35820, 128, padding_idx=0)
  (dropout): Dropout(p=0.0, inplace=False)
)

In [36]:
target_embedding.embedding_dim, target_embedding.num_embeddings

(128, 35820)

In [43]:
inputs=batch[1]
print(inputs.shape)

torch.Size([64, 49])


In [74]:
#         memory = self.encoder(sources, sources_mask)  # (batch_size, seq_len, d_model)
#         outputs, state = self.decoder(inputs, memory, memory_mask, inputs_mask)  # (batch_size, seq_len, d_model)
#译码器跑的时候，向前传播的参数是这个
inputs_embedding=target_embedding(inputs)
print(inputs_embedding.shape)
#嵌入层之后，就到译码器了输入为inputs, memory, memory_mask, inputs_mask
#这里建立这些变量

torch.Size([64, 49, 128])


In [54]:
batch_size, inputs_len = inputs.size()
import numpy as np
def subsequent_masking(x):
    # x: (batch_size, seq_len - 1)
    batch_size, seq_len = x.size()
    subsequent_mask = np.triu(np.ones(shape=(seq_len, seq_len)), k=1).astype('uint8')
    subsequent_mask = torch.tensor(subsequent_mask).to(x.device)
    subsequent_mask = subsequent_mask.unsqueeze(0).expand(batch_size, seq_len, seq_len)
    return subsequent_mask
#要不要理解这一块代码呢，今天有点发懒，后边再说吧
memory_mask = pad_masking(sources, inputs_len)
inputs_mask = subsequent_masking(inputs) | pad_masking(inputs, inputs_len)
#懂了，看这一段使用的效果，就能理解了这一块代码做了什么
print(memory_mask.shape,inputs_mask.shape)#主要看一下这个inputs_mask的形状

In [72]:
inputs_mask[0][0]

tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1], dtype=torch.uint8)

In [73]:
inputs_mask[0][1]

tensor([0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1], dtype=torch.uint8)

In [69]:
inputs_mask[0][46]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1], dtype=torch.uint8)

In [71]:
inputs[0]#和这个对比，可以知道是在哪里提前停下来了

tensor([ 2,  4,  5,  6,  7,  8,  9, 10, 11,  8, 12, 13,  8, 14, 15, 16, 17,  6,
        18, 19,  8, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [80]:
#结合这一段去理解，能看出inputs的mask就是一点点的移掉掩码的，最后到inputs的结束符为止，掩码就不移动了
print(inputs_embedding.shape,memory.shape,memory_mask.shape,inputs_mask.shape)#所需的东西都在这里了
#开始步进，进入到decoder里面去
#         inputs = self.self_attention_layer(inputs, inputs, inputs, inputs_mask, layer_cache)#自注意力
#self.self_attention_layer = Sublayer(MultiHeadAttention(heads_count, d_model, dropout_prob, mode='self-attention'), d_model)
#自注意力机制的向前传播，进入到子注意力机制里面去，不过之前研究过，就是一个q,k,v的变换
#从一个嵌入层的词向量q矩阵，点乘以同样意义的k矩阵的转置后，得到的行向量是单个字与每个字的相关性；然后q和k矩阵点乘后，再与同样意义的v矩阵点乘
#，就会得到行向量是单个字与每个字的相关性在第1维度到第128维度的特征。就相当于巧妙地转回来了，同时这个向量不单单只与自己有关，
#而是与每个词相关了。

inputs_self_att=MultiHeadAttention(heads_count=2, d_model=128, dropout_prob=0.1, mode='self-attention')(inputs_embedding,inputs_embedding,inputs_embedding,inputs_mask)
print(inputs_self_att.shape)#得到带层层递进的掩码的input输入的自注意力的词相关矩阵
#Sublayer是在做残差，对整体的理解可以跳过这一步，就走主要的机制，来试着理解这个模型的整体的变换

torch.Size([64, 49, 128]) torch.Size([64, 46, 128]) torch.Size([64, 49, 46]) torch.Size([64, 49, 49])


torch.Size([64, 49, 128])

In [98]:
print(inputs_self_att.shape,memory.shape)
#这里的mode其实没影响，因为cache是None，做一样的动作，但q,k,v的输入不一样了，这样整体的意义的变化也会有不一样
inputs_memo_att=MultiHeadAttention(heads_count=2, d_model=128, dropout_prob=0.1, mode='memory-attention')(inputs_self_att,memory,memory,memory_mask)
print(inputs_memo_att.shape)#这样德字和英字就完全关联上了。按照之前的绘图理解了一遍，算是通透

decoder_out=PointwiseFeedForwardNetwork(128, 128, 0.1)(inputs_memo_att)#非线性变化，加深网络深度
decoder_out.shape

torch.Size([64, 49, 128]) torch.Size([64, 46, 128])
torch.Size([64, 49, 128])


torch.Size([64, 49, 128])

In [111]:
#最后一步，生成最后的输出结果

generator = nn.Linear(128,target_vocabulary_size)
generated=generator(decoder_out)
print(generated.shape)

torch.Size([64, 49, 35820])


# 至此，模型部分全部结束，只需要再理解损失函数是怎么计算的，就算是从代码层面完全理解了transformer项目

有一说一，整体看下来，在没有人带着学的情况下，慢慢找材料，逐渐看懂模型，确实是个不容易的事情。但看起来以transformer作为入门模型来学习的路线还是可行的。

呼，这样就完成了整体的3/4了